<a href="https://colab.research.google.com/github/maltehueckstaedt/gerit_matching/blob/main/gerit_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GERIT PRE-Matching mit OpenAI-Embeddings

Im folgenden wird ein Notebook entworfen, dass das derzeitige Matching der HEX der Daten mit GERIT-Organisationen erleichtern soll. Im Derzeitigen Stand geht folgendermaßen vor:

Es wird von einem klassischen Cleaning der Organisations-Variable, wie sie aus den Scrapiung kommt, abgesehen, weil es im vergleich zum folgenden Vorgehen (und angesichts des Umstandes, dass wir ja die *wahren* Organisationennamen schon kennen) uneffizient und die Gefahr von Artfacten höher erscheint.

Statt eines klassischen Cleanings mit Hilfe von LLMs wird eine Funktion implementiert,die nach Entsprechungen in der Liste der GERIT-Organisationen. Dafür werden mit OpenAI Embeddings Vektor-Repräsentationen aller GERIT-Namen erstellt. Ein *„Retriever“* durchsucht diese Vektoren, um die besten Übereinstimmungen zu einem gegebenen Namen (also den Organisationsnamen aus dem Scraping) zu finden. Das Modell kann z. B. erkennen, dass „*Institut für Biologie*“ und „*Department Biologie*“ wahrscheinlich dasselbe meinen.

Für jede Organisation wird überprüft, ob es eine exakte Übereinstimmung gibt. Falls ja, wird der offizielle GERIT-Name übernommen. Falls keine exakte Übereinstimmung gefunden wird, kommt ein Fuzzy Matching zum Einsatz, bei dem ein kosinusbasierter Score berechnet wird (0 = völlig unterschiedliche Begriffe, 1 = perfekte Übereinstimmung). Ist der Score hoch genug (aktuell: > 0.5), wird der bestpassende GERIT-Treffer übernommen. Falls keine passende Entsprechung gefunden wird, bleibt der Wert NA.

Vorteile: Die Funktion spart Zeit – es werden derzeit 20.000 Zeilen und 250 unique Werte in etwa 1,5 Minuten umcodiert. Zudem ist kein manuelles Coding oder eine zusätzliche Aufbereitung der Organisationsvariablen nötig. Mit dem Score-Cutoff kann weiterhin die Konservativität des tools gesteuert werden.

Nachteile: Die Qualität der Zuordnung muss geprüft werden, und es bleibt die Frage, wie sich das neue >>Cleaning>> oder Vor-Matching auf nachfolgende Analysen auswirkt.

## Programmierung der Funktion

### Einrichtung Arbeitspfad

In einem ersten Schritt überprüfen wir unseren derzeitgen Arbeitspfad (in dem Fall das wir lokal arbeiten).

In [1]:
import os
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/gerit_matching")

### Laden und checke der Daten

Wir laden einerseits testweise der Daten (inkl. der Organisationsvariable) der HHU, andererseits die GERIT-Daten der hhu.

In [2]:
import pandas as pd
import pyreadr

# df_scraped = pd.read_csv("data\hhu_db_raw.csv")
# #df_scraped = df_scraped.sample(n=200, random_state=42)
# Einrichtung = pd.read_excel("data/Einrichtung.xlsx")

result = pyreadr.read_r("data/df_gerit.rds")
df = list(result.values())[0]  
df_gerit = df.loc[df["is_LUF_unique"] == "ja", ["Einrichtung"]]  # DataFrame statt Series

result = pyreadr.read_r("data/df_scraped.rds")
df = list(result.values())[0]  
df_scraped = df.loc[df["organisation_type_for_matching"] != "combinations", ["organisation_names_for_matching_back"]]  # DataFrame statt Series




In [3]:
df_gerit["Einrichtung"]

0      Arbeitsgemeinschaft Elektrochemischer Forschun...
1      Biologisch-Medizinisches Forschungszentrum (BMFZ)
2               Genomics und Transcriptomics Labor (GTL)
3                  Molecular Proteomics Laboratory (MPL)
4                      Center for Advanced Imaging (CAi)
                             ...                        
319    Lehrstuhl für BWL, insbes. Arbeit, Personal un...
320    Lehrstuhl für BWL, insbesondere Betriebswirtsc...
321    Lehrstuhl für BWL, insbesondere Finanzdienstle...
322            Lehrstuhl für BWL, insbesondere Marketing
323    Lehrstuhl für Volkswirtschaft, insb. Monetäre ...
Name: Einrichtung, Length: 297, dtype: object

Wir zeigen in folgenden die Spaltennamen, den Typ der Variable `organisation_names_for_matching_back` und ihre uniquen Werte an.

In [4]:
df_scraped.columns
print(df_scraped["organisation_names_for_matching_back"].dtype)
print(df_scraped["organisation_names_for_matching_back"].unique())

object
['Studiendekanat Medizinische Fakultät' 'Juristische Fakultät'
 'Sprachenzentrum' 'Wissenschaftliche Einrichtung Mathematisches Institut'
 'Wissenschaftliche Einrichtung Informatik' 'Institut für Philosophie'
 'Wissenschaftliche Einrichtung Physik' 'Institut für Romanistik'
 'Institut für Medien- und Kulturwissenschaft'
 'Institut für Kunstgeschichte' 'Institut für Experimentelle Psychologie'
 'Department Biologie' 'Institut für Modernes Japan'
 'Institut für Linguistik' 'Wirtschaftswissenschaftliche Fakultät'
 'Deutsch als Fremdsprache'
 'Institut für Pharmazeutische und Medizinische Chemie'
 'Institut für Pharmazeutische Technologie'
 'Diagnostische und Interventionelle Radiologie' 'Medizinische Fakultät'
 'Institut für Molekulare Medizin I'
 'Institut für Organische Chemie und Makromolekulare Chemie'
 'Abt. III: Deutsche Sprache und Literatur des Mittelalters'
 'Zentrum für Medizinische Mikrobiologie, Krankenhaushygiene und Virologie'
 'Abt. II: Neuere Dt. Literatur - Lehrstu

Unabhängig davon, welcher Typ die Variable `organisation_names_for_matching_back` tatsächlich ist, definieren wandeln wie die Varibale in einen String.

Anschließend erzeugen wir mit `set()` eine leere Menge `unique_values`. `set()` erzeugt eine ungeordnete Sammlung von einzigartigen Elementen. Sets entfernen automatisch doppelte Werte und bieten effiziente Methoden zum Hinzufügen, Entfernen und Überprüfen von Elementen.


Wir nehmen dann den Datensatz `df_scraped` und entfernen alle `NaN`. Anschließend wird für jeden verbleibenden Wert die Funktion `apply()` aufgerufen. Innerhalb der Funktion wird der Text jedes Wertes an den Semikolons geteilt, sodass eine Liste von Werten entsteht. Diese Werte werden dann bereinigt, indem führende und nachstehende Leerzeichen entfernt werden und leere oder mit "NA" (unabhängig von der Groß-/Kleinschreibung) markierte Werte ausgeschlossen werden. Alle bereinigten, nicht leeren und nicht "NA"-Werte werden anschließend dem `unique_values`-Set hinzugefügt, wobei nur einzigartige Werte beibehalten werden.

Abschließend wird zur Information die Anzahl uniquer Organisationen sowie die Merkmale ausgegeben.

In [5]:
import numpy as np

df_scraped["organisation_names_for_matching_back"] = df_scraped["organisation_names_for_matching_back"].astype(str)

unique_values = set()
df_scraped["organisation_names_for_matching_back"].dropna().apply(lambda x: unique_values.update(
    [val.strip() for val in x.split(";") if val.strip() and val.strip().upper() != "NA"]
))

# Ergebnis ausgeben
print(f"Anzahl einzigartiger Werte: {len(unique_values)}")
print(unique_values)

Anzahl einzigartiger Werte: 247
{'Abt. IV: Theorie und Praxis (Mündlichkeit)', 'Institut für Biochemie der Pflanzen', 'Dekanatsbüro Phil. Fakultät', 'Institut für Geschichte, Theorie und Ethik der Medizin', 'Zentrum für Pharmakologie und Toxikologie', 'Neurobiologie', 'Quantitative und Theoretische Biologie CEPLAS', 'Institut für Neuropathologie', 'Klinik für Anästhesiologie', 'Institut für Klinische Diabetologie', 'Zentrale Einrichtung für Tierforschung und wiss. Tierschutzaufgaben', 'Molekulare Evolution', 'Institut für Biochemie und Molekularbiologie I', 'Abteilung III Alte Geschichte', 'Soziologie III', 'Molekulare Physiologie', 'Entwicklungs- und Molekularbiologie der Tiere', 'Musikwissenschaftliches Institut der Robert-Schumann-Hochschule Düsseldorf', 'Wissenschaftliche Einrichtung Mathematisches Institut', 'Lehrstuhl für Betriebswirtschaftslehre, insbes. Betriebswirtschaftliche Steuerlehre', 'Lehrstuhl für Betriebswirtschaftslehre, insbes. Controlling und Accounting', 'Lehrstuhl

### Lade openAI-API

Für das Matching der Organisationsnamen, die aus dem Scraping kommen mit denen der GERIT-Organisationen benötigen wir Embeddings. Dies Embeddings werden mit Hilfe der openAI-API erzeugt. Aus diesem Grund wird der openAI-API-Key im folgenden aus dem Enviorment geladen.


In [6]:
from dotenv import load_dotenv

load_dotenv()

True

### Embeddings generieren

Im folgenden wird die OpenAI-API verwendet, um Embeddings der GERIT-Organisationen zu erzeugen und mit hilfe von Langchain eine In-Memory-Vektordatenbank für GERIT-Organisationen zu erstellen.

In einem ersten Schritt wird das OpenAI-Embedding-Modell `text-embedding-3-large` initialisiert. Danach wird eine InMemoryVectorStore-Datenbank erstellt, die diese Vektoren speichert.

Nun werden die GERIT-Organisationen aus dem Pandas-DataFrame (`df_gerit["Einrichtung"]`) in `Document`-Objekte umgewandelt. Diese Dokumente enthalten die Namen der Einrichtungen als `page_content`. Schließlich werden diese Dokumente mit ihren Embeddings in die Vektordatenbank eingefügt. Dadurch können später Ähnlichkeitsabfragen durchgeführt werden, um Organisationen mit ähnlichen Namen oder thematischen Schwerpunkten zu finden.

In [7]:
import openai
import pandas as pd
from typing import List
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_core.runnables import chain
from langchain.agents.agent_toolkits import create_retriever_tool

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

docs = [Document(page_content=einrichtung) for einrichtung in df_gerit["Einrichtung"].tolist()]
vector_store.add_documents(docs)

['f14f7328-f395-4244-a2c3-254d712ff8f5',
 '13b906a4-d996-432c-8107-dfbf8aa036ef',
 '043cd84a-ce15-45ef-9fb1-9f805dbf9778',
 '8a1a7c70-675b-48e2-9d7f-71bcb8c9ade0',
 'ee0b2aee-5ed5-46b0-8b6c-0543ddb31490',
 '00a71a83-b9fe-46f0-91a7-2784bba5c6b1',
 '4651d70e-059f-4689-9111-cd926a7b7300',
 'ea094dca-3bcf-43b1-a6cb-ff16a90142fa',
 '5722d33d-1566-425d-a32f-f764cb4598c1',
 'f9ebbfea-e155-4e4c-9673-774af2445327',
 'ed5e6c56-51df-4c77-b1a3-d68d3468b9ac',
 '27ff7469-b3a2-42dd-a507-ddf892836173',
 '7b5b248e-637e-47b8-9c35-a5e274b054fe',
 '8f2dd337-d619-48b6-b98d-a1ee44a5c354',
 'fda146ab-9725-4032-8f71-6612b5a54884',
 '6e610a25-d95c-43b1-bb59-223a8d3fa372',
 '3842b1d2-8b59-4b9f-bb1d-d44ae935c800',
 'fb1e275d-244c-4c0c-bfce-4895a31e34ce',
 '5783a005-d37b-4134-9b5c-acb113ee36da',
 'd43b20e8-dd0f-4a5c-8c53-79f616de7880',
 '6c2f382a-8a9b-4228-aca9-18ac45a0e48e',
 '6f8095a9-34b2-4810-b195-d016465854ec',
 '8e73a14a-f323-4892-b93a-6ba4f8b43c03',
 'd806a891-72ac-45cb-80f8-afab82b754bc',
 '346d5e79-2800-

### Erzeuge Retriever

Im weiteren definieren wir eine Retriever-Funktion, die anhand einer Suchanfrage ähnliche Dokumente aus dem oben definierten Vektor-Store findet und ihre Ähnlichkeitsscores zurückgibt. Dazu wird die Funktion `retriever` mit `@chain` als Teil einer Verarbeitungskette registriert. Die Suche erfolgt mit `vector_store.similarity_search_with_score(query, k=5)`, wodurch die fünf ähnlichsten Dokumente ermittelt werden. Falls Ergebnisse vorhanden sind, werden sie in Dokumente und Scores aufgeteilt. Jedes gefundene Dokument erhält seinen Ähnlichkeitsscore als Metadaten.  

Zusätzlich wird eine Beschreibung für den Retriever erstellt, die erklärt, dass die Funktion nach gültigen Eigennamen basierend auf einer ungefähren Eingabe sucht und die Ähnlichkeitsscores zurückliefert. Falls keine passende Entsprechung gefunden wird, wird stattdessen "NA" zurückgegeben. Schließlich wird mit `create_retriever_tool` ein Tool erstellt, das die definierte Retriever-Funktion mit der angegebenen Beschreibung verbindet und als `retriever_tool` gespeichert wird.

In [8]:
# Definiere den Retriever als Chain mit Scores
@chain
def retriever(query: str) -> List[Document]:
    results = vector_store.similarity_search_with_score(query, k=5)
    docs, scores = zip(*results) if results else ([], [])

    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score  # Füge den Score als Metadaten hinzu

    return list(docs)

# Beschreibung für den Retriever
description = (
    "Sucht nach gültigen Eigennamen basierend auf einer ungefähren Eingabe und gibt die Ähnlichkeitsscores zurück. "
    "Falls keine passende Entsprechung gefunden wird, wird 'NA' zurückgegeben."
)

# Retriever-Tool erstellen
retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns_with_score",
    description=description,
)

#### Teste Retriever

Wir testen im folgenden den Retriever auf Funktionalität.

In [9]:
query = "Physikfdalische Chemie"
results = retriever.invoke(query)

# Ergebnisse anzeigen
for doc in results:
    print(f"Text: {doc.page_content}, Score: {doc.metadata['score']}")


Text: Institut für Physikalische Chemie, Score: 0.7465456896876095
Text: Lehrstuhl für Festkörperphysik, Score: 0.6288085595609698
Text: Institut für Theoretische Chemie, Score: 0.6284249496503709
Text: Lehrstuhl für Molekulare Physikalische Chemie (MPC), Score: 0.6104274537375814
Text: Lehrstuhl für Physik der weichen Materie, Score: 0.5993830425468181


#### Erzeuge Matching-Funktion

Die Funktion `match_unique_organisations` führt eine unscharfe Suche für Organisationsnamen durch und erstellt eine detaillierte Matching-Tabelle. Sie verarbeitet eine `pandas.df` mit Organisationen, wobei Organisationen, die durch `";"` getrennt sind, einzeln betrachtet werden.

Zunächst wird ein leeres Rekodierungs-Dictionary (`recode_dict`) und eine Liste für die Matching-Ergebnisse (`matching_data`) erstellt. Für jede Organisation im Eingangs-Data Frame wird geprüft, ob sie durch `";"` getrennte Einträge enthält, die dann separat verarbeitet werden.

Für jede Organisation wird der `retriever` verwendet, um eine unscharfe Suche durchzuführen. Falls das beste gefundene Ergebnis einen Ähnlichkeitsscore von mindestens `0.65` hat, wird es als Match übernommen, andernfalls bleibt die ursprüngliche Organisation bestehen. Das Ergebnis wird mit einer entsprechenden Matching-Art (`Fuzzy` oder `Keine Übereinstimmung`) und dem Score gespeichert.

Das `recode_dict` enthält die ursprünglichen Organisationen als Schlüssel und die zugeordneten Werte als Zeichenketten mit `"; "` getrennten gematchten Namen. Der `matching_df` speichert detaillierte Informationen über das Matching, einschließlich des ursprünglichen Wertes, des gematchten GERIT-Wertes, des Matching-Typs und des Scores.

Am Ende gibt die Funktion das Rekodierungs-Dictionary und den DataFrame mit den Matching-Details zurück.

In [10]:
import pandas as pd

def match_unique_organisations(org_df, retriever):
    """
    Funktion, die nur einzigartige Werte der Organisationen verarbeitet und eine detaillierte Matching-Tabelle erstellt.
    
    :param org_df: Pandas Series mit den ursprünglichen Organisationen
    :param retriever: Retriever zur unscharfen Suche
    :return:
        - recode_dict: Dictionary mit den rekodierten Werten (Originalwert → gematchter Wert)
        - matching_df: DataFrame mit Matching-Details (ursprünglicher Wert, gematchter Wert, Matching-Art, Score)
    """
    print(f"Anzahl einzigartiger Organisationen in den Eingabedaten: {len(org_df)}")

    recode_dict = {}
    matching_data = []  # Liste für die Matching-Ergebnisse

    for org in org_df:
        orgs = [o.strip() for o in org.split(';')]  # Aufteilen bei ";", Leerzeichen entfernen
        matched_orgs = []
        match_details = []  # Speichert Match-Typ und Score für jede Organisation in einer Zeile

        for o in orgs:
            match_type = "Keine Übereinstimmung"
            score = None
            matched_value = None

            # Unscharfe Suche über den Retriever
            results = retriever.invoke(o)

            if results and results[0].metadata.get("score", 0) >= 0.65:
                matched_value = results[0].page_content
                score = results[0].metadata["score"]
                match_type = "Fuzzy"
                print(f"Fuzzy Match gefunden: '{o}' → '{matched_value}' (Score: {score:.2f})")
            else:
                print(f"Kein passender Match für '{o}' gefunden.")

            # Speichern des Matches
            matched_orgs.append(matched_value if matched_value else o)  # Falls kein Match, Original behalten
            match_details.append({
                "Ursprünglicher Wert": o,
                "Gematchter GERIT-Wert": matched_value,
                "Matching-Art": match_type,
                "Score": score
            })

        # Rekodierungs-Dictionary speichern
        recode_dict[org] = "; ".join([m for m in matched_orgs if m]) if matched_orgs else None

        # Matching-Details zur Liste hinzufügen
        matching_data.extend(match_details)

    # DataFrame aus den Matching-Daten erstellen
    matching_df = pd.DataFrame(matching_data)

    return recode_dict, matching_df


In [11]:
recode_dict, matching_df = match_unique_organisations(unique_values, retriever)


Anzahl einzigartiger Organisationen in den Eingabedaten: 247
Fuzzy Match gefunden: 'Abt. IV: Theorie und Praxis (Mündlichkeit)' → 'Abteilung IV: Lehrstuhl Theorie und Praxis mündlicher und schriftlicher Kommunikation' (Score: 0.67)
Fuzzy Match gefunden: 'Institut für Biochemie der Pflanzen' → 'Institut für Biochemie der Pflanzen' (Score: 1.00)
Kein passender Match für 'Dekanatsbüro Phil. Fakultät' gefunden.
Fuzzy Match gefunden: 'Institut für Geschichte, Theorie und Ethik der Medizin' → 'Institut für Geschichte, Theorie und Ethik der Medizin' (Score: 1.00)
Fuzzy Match gefunden: 'Zentrum für Pharmakologie und Toxikologie' → 'Institut für Pharmakologie' (Score: 0.78)
Fuzzy Match gefunden: 'Neurobiologie' → 'Angewandte Neurobiologie' (Score: 0.78)
Fuzzy Match gefunden: 'Quantitative und Theoretische Biologie CEPLAS' → 'Institut für Quantitative und Theoretische Biologie' (Score: 0.74)
Fuzzy Match gefunden: 'Institut für Neuropathologie' → 'Institut für Neuropathologie' (Score: 1.00)
Fuzzy

#### Erzeuge Applyer-Funktion

Die Funktion `apply_matched_organisations` verwendet das zuvor erstellte Rekodierungs-Dictionary (`recode_dict`), um die ursprünglichen Organisationsnamen im Pandas-Data Frame (`org_df`) durch die gematchten Werte zu ersetzen.

Zunächst wird eine innere Funktion `map_multiple_orgs` definiert, die für jeden Eintrag in der Spalte ausgeführt wird. Falls der Wert `NaN` ist, wird `None` zurückgegeben. Falls der Wert mehrere Organisationen enthält, werden diese anhand des Trennzeichens `";"` aufgeteilt und bereinigt. Für jede einzelne Organisation wird geprüft, ob sie im `recode_dict` vorhanden ist. Falls ja, wird der gematchte Wert ersetzt, ansonsten bleibt der Originalwert bestehen.

Nach der Rekodierung werden leere Werte aus der Liste entfernt. Falls am Ende noch gültige Organisationen vorhanden sind, werden sie mit `"; "` wieder zusammengeführt. Falls keine gültigen Werte übrig bleiben, wird `None` zurückgegeben.

Am Ende wird die gesamte Spalte mithilfe von `.apply()` transformiert, sodass alle Organisationsnamen durch ihre gematchten Werte ersetzt werden.


In [14]:
def apply_matched_organisations(org_df, recode_dict):
    """
    Wendet das zuvor berechnete Matching Dictionary auf die originale Spalte an,
    indem alle Organisationen innerhalb eines Eintrags korrekt ersetzt werden.

    :param org_df: Pandas Series mit den ursprünglichen Organisationen
    :param recode_dict: Dictionary mit den rekodierten Werten (Originalwert → gematchter Wert)
    :return: Pandas Series mit den rekodierten Organisationen
    """
    def map_multiple_orgs(org):
        if pd.isna(org):
            return None  # Fehlende Werte beibehalten

        org_list = [o.strip() for o in org.split(';')]  # Mehrere Organisationen aufteilen
        matched_list = [recode_dict.get(o, o) for o in org_list]  # Falls kein Match gefunden wird, Originalwert behalten
        matched_list = [m for m in matched_list if m]  # Leere Werte entfernen

        return "; ".join(matched_list) if matched_list else None  # Falls keine Matches, None zurückgeben

    return org_df.apply(map_multiple_orgs)


In [15]:
df["gerit_organisation"] = apply_matched_organisations(df_scraped["organisation_names_for_matching_back"], recode_dict)

# Rückspielen an urspürnglichen df

Im folgenden werden die Ergebnisse schließlich an den `df` gematcht. Dies sind die Variablen `Score` sowie `match_type`, `gerit_organisation`.

Zunächst wird dafür aus `recode_dict` ein neuer `DataFrame` (`recode_df`) erstellt, in dem die ursprünglichen Organisationsnamen als Index dienen und den gematchten GERIT-Wert enthalten. Der Index wird zurückgesetzt, sodass eine Spalte `"organisation_names_for_matching_back"` entsteht.  

Zusätzlich wird aus dem `matching_df` eine reduzierte Version (`matching_info`) erstellt, die nur die relevanten Spalten enthält: den ursprünglichen Wert (`"Ursprünglicher Wert"`), den Score und die Matching-Art (`"Matching-Art"`). Die Spalten werden dabei umbenannt, damit sie einheitlich zum späteren Merge passen.  

Danach wird der ursprüngliche `df` um diese neuen Informationen ergänzt. Dabei werden zuerst die gematchten Organisationsnamen (`recode_df`) und anschließend die Matching-Scores sowie die Matching-Art (`matching_info`) über einen `left join` anhand der Spalte `"organisation_names_for_matching_back"` hinzugefügt.  
Falls durch den Merge neue Spalten mit dem Suffix `"_new"` entstehen, werden die ursprünglichen Spalten (`"gerit_organisation"`, `"score"`, `"match_type"`) durch diese neuen Werte ersetzt. Falls eine neue Spalte existiert, wird sie in `df[col]` überschrieben und anschließend aus `df` entfernt, sodass keine unnötigen `_new`-Spalten übrig bleiben.

In [16]:
# DataFrame aus dem recode_dict erstellen
recode_df = pd.DataFrame.from_dict(recode_dict, orient="index", columns=["gerit_organisation"])
recode_df.reset_index(inplace=True)
recode_df.rename(columns={"index": "organisation_names_for_matching_back"}, inplace=True)

# Auch das Matching-DF nutzen, um Score und Matching-Art (umbenannt in match_type) anzufügen
matching_info = matching_df[['Ursprünglicher Wert', 'Score', 'Matching-Art']].drop_duplicates()
matching_info.rename(columns={'Ursprünglicher Wert': 'organisation_names_for_matching_back',
                              'Matching-Art': 'match_type'}, inplace=True)

# Merge mit df, dabei vorhandene Spalten ersetzen
df = df.merge(recode_df, on="organisation_names_for_matching_back", how="left", suffixes=("", "_new"))
df = df.merge(matching_info, on="organisation_names_for_matching_back", how="left", suffixes=("", "_new"))

# Existierende Spalten durch neue Werte ersetzen, falls vorhanden
for col in ["gerit_organisation", "score", "match_type"]:
    new_col = col + "_new"
    if new_col in df.columns:
        df[col] = df[new_col]
        df.drop(columns=[new_col], inplace=True)

In [17]:
df.columns

Index(['organisation_names_for_matching_back', 'organisation',
       'organisation_alternativ', 'cleaned', 'cleaned_alternativ',
       'scraped_ID', 'organisation_type_for_matching', 'no_courses',
       'percent_courses', 'no_courses_from_2017', 'percent_courses_from_2017',
       'matched', 'match_type', 'gerit_organisation', 'gerit_cleaned',
       'gerit_studienbereich', 'gerit_faechergruppe', 'gerit_ID', 'name_rds',
       'latest_scraped_semester_is', 'this_matching_is', 'Score'],
      dtype='object')